<a href="https://colab.research.google.com/github/felipeoliveiralps/Analise-Sentimento-Ecommerce/blob/master/notebooks/NLP/AnaliseSentimentoBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Carregando dataset já tratado**

In [ ]:
import pandas as pd

dataSemStopWords = pd.read_csv('../../datasets/dataSemStopWords.csv', engine='python')


In [ ]:
pip install tensorflow

**2.1 Divisão Estratificada do DataSet: Treino, Validação e Teste**

* Para preparar a modelagem, irei dividir os dados em três conjuntos: treino (80%), validação (10%) e teste (10%). Essa separação é fundamental para treinar o modelo, otimizar seus parâmetros e, por fim, realizar uma avaliação final e imparcial.

* A divisão será feita de forma estratificada utilizando a coluna feedback como base. Isso garante que a proporção de classes (positivas e negativas) seja exatamente a mesma nos três conjuntos, o que é crucial para a confiabilidade do treinamento e da avaliação do modelo.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, TFBertForSequenceClassification, create_optimizer
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import seaborn as sns
import matplotlib.pyplot as plt
import random
import os

seed_value = 49

os.environ['PYTHONHASHSEED'] = str(seed_value)

random.seed(seed_value)

np.random.seed(seed_value)

tf.random.set_seed(seed_value)

# --- Passo 1: Primeira Divisão (Treino / Resto) ---
# Vamos separar 70% dos dados para o treino e deixar 30% para o "resto".
train_df, temp_df = train_test_split(
    dataSemStopWords,
    test_size=0.3,
    random_state=seed_value,
    stratify=dataSemStopWords['feedback']
)

# --- Passo 2: Segunda Divisão (Validação / Teste) ---
val_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,
    random_state=seed_value,
    stratify=temp_df['feedback']
)


**2.3 Carregando modelo e Tokenizer (BERTIMBAU)**
* Para esta tarefa de classificação de texto o BERTIMBAU, um modelo pré-treinado.

* A utilização de um modelo como o BERTIMBAU, em vez de treinar um do zero, nos permite transferir o "conhecimento" linguístico que ele já possui para a nossa tarefa específica. Isso resulta em um desempenho significativamente superior e um tempo de treinamento muito menor para alcançar alta acurácia.



In [ ]:
model_name = "neuralmind/bert-base-portuguese-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=3)

train_df['texto_lematizado'] = train_df['texto_lematizado'].astype(str)
val_df['texto_lematizado'] = val_df['texto_lematizado'].astype(str)
test_df['texto_lematizado'] = test_df['texto_lematizado'].astype(str)


**2.4 Preparando Dados**

In [ ]:
train_encodings = tokenizer(train_df['texto_lematizado'].tolist(), truncation=True, padding=True, max_length=150, return_tensors='tf')
val_encodings = tokenizer(val_df['texto_lematizado'].tolist(), truncation=True, padding=True, max_length=150, return_tensors='tf')
test_encodings = tokenizer(test_df['texto_lematizado'].tolist(), truncation=True, padding=True, max_length=150, return_tensors='tf')

train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_df['feedback'].values))
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_encodings), val_df['feedback'].values))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), test_df['feedback'].values))

**2.5 Criando o peso das classes**
* Durante a análise, percebi que nosso conjunto de treino possui um desnível entre as categorias de feedback, ou seja, uma classe tem muito mais amostras que a outra. Se treinarmos o modelo com esses dados, ele pode se tornar "preguiçoso", aprendendo a prever apenas a classe majoritária e ignorando a minoritária, resultando em um desempenho ruim na prática.

* Para corrigir esse viés, irei calcular e aplicar pesos de classe. Essa técnica modifica a função de perda do modelo, aplicando uma penalidade maior quando ele comete um erro na classe minoritária.


In [ ]:
y_train_labels = train_df['feedback'].values
class_labels = np.unique(y_train_labels)
class_weights_array = compute_class_weight(class_weight='balanced', classes=class_labels, y=y_train_labels)
class_weight_dict = dict(zip(class_labels, class_weights_array))

print(f"Classes: {class_labels}")
print(f"Pesos das Classes Calculados: {class_weights_array}")
print(f"Dicionário de Pesos para Keras: {class_weight_dict}")

**Configurando hiperParametros**

* Com os dados prontos, agora vou configurar os parametros para a modelagem. Irei configurar o batch_size como 16, que define quantos exemplos o modelo verá antes de atualizar seus pesos. O número de epochs será 2, o que significa que o modelo irá percorrer todo o conjunto de dados de treino duas vezes, além do learning rate scheduler com warmup.




In [ ]:
batch_size = 16
epochs = 3

train_batches = train_dataset.shuffle(len(train_df), seed=seed_value, reshuffle_each_iteration=False).batch(batch_size)
val_batches = val_dataset.batch(batch_size)
test_batches = test_dataset.batch(batch_size)

steps_per_epoch = len(train_df) // batch_size
if len(train_df) % batch_size != 0:
    steps_per_epoch += 1
total_train_steps = steps_per_epoch * epochs

optimizer, lr_schedule = create_optimizer(
    init_lr=2e-5,
    num_train_steps=total_train_steps,
    num_warmup_steps=int(0.1 * total_train_steps),
    weight_decay_rate=0.01
)

**Compilando Modelo!!**

In [ ]:
model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

**Treinando Modelo**

In [ ]:
print("Iniciando o treinamento do BERTimbau com class_weight...")

history = model.fit(
    train_batches,
    validation_data=val_batches,
    epochs=epochs,
    class_weight=class_weight_dict,
    verbose=1
)
print("Treinamento concluído!")

**Resultados**

In [ ]:
print("\n--- Avaliação no Conjunto de Teste (modelo da última epoch) ---")
result = model.evaluate(test_batches)
print(f"Loss no Teste: {result[0]:.4f}")
print(f"Acurácia no Teste: {result[1] * 100:.2f}%")

preds = model.predict(test_batches)
pred_classes = np.argmax(preds.logits, axis=1)

print("\nRelatório de Classificação:")
print(classification_report(
    list(test_df['feedback'].values),
    pred_classes,
    target_names=['Negativo', 'Neutro', 'Positivo']
))

cm = confusion_matrix(list(test_df['feedback'].values), pred_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Negativo', 'Neutro', 'Positivo'],
            yticklabels=['Negativo', 'Neutro', 'Positivo'])
plt.xlabel('Previsto')
plt.ylabel('Real')
plt.title('Matriz de Confusão - BERTimbau')
plt.show()